In [2]:
# Technical libraries
import cvxpy as cp
import numpy as np
import numpy.matlib
import numpy.random
from scipy import linalg, special
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy 
import scipy.fftpack
# from proximal import ProxUtils
import proximal
from iplabs import IPLabViewer as viewer
import matplotlib
%matplotlib widget

# Proximal Operators for Nonnegative Inverse Problems

Study of the combination of different image regularizers with nonnegativity constraints.

# Index

1. [Introduction](#Intro)
    1. [Proximal Operator](#Prox)
    2. [Nonnegativity Function](#Nonneg)
2. [Norms](#2.-Norms)
    1. [$\ell_p$](#2.A-Lp)
    2. [$\ell_p^p$](#2.B-Lp^p)
3. [Group Sparsity](#3.-Group-Sparsity)
    1. [1D Group Sparsity](#3.A-1D-Group-Sparsity)
    2. [2D Group Sparsity](#3.B-2D-Group-Sparsity)
4. [Total Variation](#4.-Spectral-Norm)
    1. [1D Total Variation](#4.A-1D-Total-Variation)
    2. [2D Total Variation](#4.B-2D-Total-Variation)
5. [Norms on Eigendecomposition](#5.-Norms-on-Eigendecomposition)
    1. [$\ell_p$](#5.A-Lp)
    2. [$\ell_p^p$](#5.B-Lp^p)
6. [Hessian Schatten](#6.-Hessian-Schatten-Norm)
6. DCT

# <a name="Intro"></a>1. Introduction

The goal of the present notebook is to find how of common image regularizers combine with nonnegativity through the proximal operator. We will be studying regularizers likethe $\mathrm{L}^1$ and the $\mathrm{L}^2$ norm, and group sparsity.

## <a name="Prox"></a>1.A. Proximal Operator

The proximal operator of a function $f$ is:

$$\mathrm{prox}_f(v) = \arg \min_x(f(x)+\frac{1}{2\lambda}||x - v||_2^2)$$

This means that it will optimize an input vector $v$ with respect to a function, but adding the constraint that the result has to be *somewhat close* (by the minimization of the second term, and with *somewhat* parametrized by $\lambda$) to the original. 

The interest of the project is to see how common image regularizers combine with nonnegativity constraints. In particular, we will see wether:
$$\mathrm{prox}_{f} = \mathrm{prox}_{\delta_{\rm I\!R_+^N}}(\mathrm{prox}_{\Re})$$
or 
$$\mathrm{prox}_{f} = \mathrm{prox}_{\Re}(\mathrm{prox}_{\delta_{\rm I\!R_+^N}})$$
where $f = \delta_{\rm I\!R_+^N} + \Re$,
for several regularizers, starting by the know cases of $\mathrm{L^1}$ and $\mathrm{L^2}$ norms. 

In some cases for which the $\mathrm{prox}$ is a point-wise operation, a plot of $v_i$ vs $x_i$ can be made, where $v_i$ is the $i^{th}$ element of $v$. In this cases, the previous equations can be provedor discarded analitically.  

## <a name="Nonneg"></a>1.B. Nonnegativity Function

The nonnegativity function $\delta$ is defined as:

$$\delta_{\rm I\!R_+^N} =
\begin{cases}
        0 \mathrm{ if } x \in \rm I\!R_+^N \\
        + \inf \mathrm{ if } x\notin \rm I\!R_+^N
     \end{cases}$$
     
Given the definition of the proximal operator, it is immediate to see that to solve the proximal operator, we have as constraint $x\in \rm I\!R_+^N$. As such, the first term vanishes (since $x\in \rm I\!R_+^N$, by definition $f(x) = 0$). As such, we arrive to:

$$prox_{\rm I\!R_+^N}(v) = \arg \min_{x\in \rm I\!R_+^N}\left(\frac{1}{2}||x - v||_2^2\right)$$

Given that $v \in \rm I\!R_+^N$, then the $x\in \rm I\!R_+^N$ that minimizes the prox operator is simply the $x$ closest to $v$, but yet inside the domain imposed by the indicator function (indicator of nonnegativity). It is useless to include the parameter $\lambda$ in this proximal, as the minimization of squares is the only term present.

# 2. Norms
[Back to Index](#Index)

In this section we will prove the equations provided on [1.A](#Prox), for the $\mathrm{L}^1$ norm. It is a part from the family of the $L^p$ norms where $p = 1$, is defined for a vector $x$ as:
$$\|\mathbf{x}\|_p = (\sum_{i = 1}^n \mathbf{x}^p)^\frac{1}{p} $$

It follows that the $L^1$ norm of a vector is simply the sum of its components. It is a common image regularizer because it enforces sparsity, a well known property of natural images. 

### 2.A Lp
[Back to Index](#Index)

In [2]:
for p in [1, 2, 3]:
    print(f'||p norm where p = {p}')
    if p == 5:
        proximal.evaluate(proximal.Lp_prox, proximal.Lp_plus_nonneg_prox, n=1, size=(20, 20), mean=0, sigma=1, lamb=0.5, additional_params=[p], err_thr=1e-4, plot=True)
        continue
    proximal.evaluate(proximal.Lp_prox, proximal.Lp_plus_nonneg_prox, n=1, size=(50, 50), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[p], err_thr = 1e-4)
             

||p norm where p = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 4.296e-07
Average absolute error: 2.745e-08

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 4.296e-07
Average absolute error: 2.745e-08

||p norm where p = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.316e-02
Average absolute error: 1.584e-03

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 2.247e-05
Average absolute error: 2.630e-06

||p norm where p = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.050e-02
Average absolute error: 5.961e-04

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 1.026e-07
Average absolute error: 9.790e-10



### 2.B Lp^p
[Back to Index](#Index)

In [14]:
for p in [1, 2, 3, 5, 8]:
    print(f'||p^p norm where p = {p}')
    if p == 5:
        proximal.evaluate(proximal.Lp_prox, proximal.Lp_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=1, lamb=0.5, additional_params=[p, True], err_thr=1e-4, plot=True)
        continue
    proximal.evaluate(proximal.Lp_prox, proximal.Lp_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[p, True], err_thr = 1e-3)

||p^p norm where p = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 2.001e-07
Average absolute error: 5.571e-09

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 2.001e-07
Average absolute error: 5.571e-09

||p^p norm where p = 2
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.276e-04
Average absolute error: 1.211e-05

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 1.670e-04
Average absolute error: 1.953e-05

||p^p norm where p = 3
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 2.899e-05
Average absolute error: 6.574e-07

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 1.829e-06
Average absolute error: 5.253e-08

||p^p norm where p = 5
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 6.091e-06
Average absolute error: 1.581e-07

prox_reg(prox_nonneg(v)) SEEMS equal

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

||p^p norm where p = 8
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 7.004e-07
Average absolute error: 3.266e-08

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 7.914e-07
Average absolute error: 2.953e-08



## 3. Group Sparsity
[Back to Index](#Index)

The group sparsity operator is denoted by $\operatorname{G}$

### 3.A 1D Group Sparsity

$$\|\operatorname{G}x\|_{p, q}$$

In [2]:
for p in [1, 2, 3]:
    for q in [1, 2, 3]:
        print(f'||p norm where p = {p}, q = {q}')
        if p == 1 and q == 2:
            proximal.evaluate(proximal.GS_1D_prox, proximal.GS_1D_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=1, lamb=0.5, additional_params=[5, p, q], err_thr=1e-3, plot=True)
            continue
        proximal.evaluate(proximal.GS_1D_prox, proximal.GS_1D_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[5, p, q], err_thr = 1e-3)

||p norm where p = 1, q = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.997e-05
Average absolute error: 1.498e-06

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 1.997e-05
Average absolute error: 1.498e-06

||p norm where p = 1, q = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.753e-01
Average absolute error: 2.462e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 4.518e-05
Average absolute error: 3.400e-06

Plotting example


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

||p norm where p = 1, q = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 2.658e-01
Average absolute error: 3.159e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 2.572e-05
Average absolute error: 1.212e-06

||p norm where p = 2, q = 1
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 3.632e-01
Average absolute error: 3.694e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 9.455e-05
Average absolute error: 5.840e-06

||p norm where p = 2, q = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.461e-01
Average absolute error: 1.541e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 1.549e-04
Average absolute error: 1.210e-05

||p norm where p = 2, q = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.648e-01
Average absolute error: 1.270e-02

prox_reg(prox_no

$$\|\operatorname{G}x\|_{p, q}^{p, q}$$

In [3]:
for p in [1, 2, 3]:
    for q in [1, 2, 3]:
        print(f'||p norm where p = {p}, q = {q}')
        if p == 1 and q == 2:
            proximal.evaluate(proximal.GS_1D_prox, proximal.GS_1D_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=1, lamb=0.5, additional_params=[5, p, q, True], err_thr=1e-3, plot=True)
            continue
        proximal.evaluate(proximal.GS_1D_prox, proximal.GS_1D_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[5, p, q, True], err_thr = 1e-3)

||p norm where p = 1, q = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 2.270e-04
Average absolute error: 1.064e-05

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 2.347e-04
Average absolute error: 9.615e-06

||p norm where p = 1, q = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 6.256e-01
Average absolute error: 3.061e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 3.793e-05
Average absolute error: 1.658e-06

Plotting example


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

||p norm where p = 1, q = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 7.107e-01
Average absolute error: 3.904e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 3.052e-05
Average absolute error: 1.347e-06

||p norm where p = 2, q = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.091e-04
Average absolute error: 3.829e-06

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 8.551e-05
Average absolute error: 3.937e-06

||p norm where p = 2, q = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 2.204e-01
Average absolute error: 2.632e-02

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 3.887e-05
Average absolute error: 2.737e-06

||p norm where p = 2, q = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 3.738e-01
Average absolute error: 3.720e-02

prox_reg(prox_no

### 3.B 2D Group Sparsity
[Back to Index](#Index)

$$\|\operatorname{G}x\|_{p, q}$$

In [4]:
for p in [1, 2]:
    for q in [1, 2, 3]:
        print(f'||p norm where p = {p}, q = {q}')
        if p == 1 and q == 2:
            proximal.evaluate(proximal.GS_prox, proximal.GS_plus_nonneg_prox, n=10, size=(30, 30,), mean=0, sigma=1, lamb=0.5, additional_params=[(5, 5), p, q], err_thr=1e-3, plot=True)
            continue
        proximal.evaluate(proximal.GS_prox, proximal.GS_plus_nonneg_prox, n=10, size=(30, 30,), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[(5, 5), p, q], err_thr = 1e-3)

||p norm where p = 1, q = 1
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 4.848e-01
Average absolute error: 1.506e-03

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 1.381e-04
Average absolute error: 2.898e-07

||p norm where p = 1, q = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 3.258e-01
Average absolute error: 1.555e-03

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 7.268e-05
Average absolute error: 1.870e-07

Plotting example as image.


Button(description='Show Widgets', style=ButtonStyle())

||p norm where p = 1, q = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 4.247e-01
Average absolute error: 1.576e-03

prox_reg(prox_nonneg(v)) SEEMS equal to prox(reg + nonneg)
Max absolute error: 9.748e-05
Average absolute error: 2.157e-07

||p norm where p = 2, q = 1
FATAL: Cannot solve SDPs with > 2x2 matrices without linked blas+lapack libraries
Install blas+lapack and re-compile SCS with blas+lapack library locations
ERROR: init_cone failure
Failure:could not initialize work


SolverError: Solver 'SCS' failed. Try another solver, or solve with verbose=True for more information.

$$\|\operatorname{G}x\|_{p, q}^{p, q}$$

In [5]:
for p in [1, 2]:
    for q in [1, 2, 3]:
        print(f'||p norm where p = {p}, q = {q}')
        if p == 1 and q == 2:
            proximal.evaluate(proximal.GS_prox, proximal.GS_plus_nonneg_prox, n=10, size=(30, 30,), mean=0, sigma=1, lamb=0.5, additional_params=[(5, 5), p, q, True], err_thr=1e-3, plot=True)
            continue
        proximal.evaluate(proximal.GS_prox, proximal.GS_plus_nonneg_prox, n=10, size=(30, 30,), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[(5, 5), p, q, True], err_thr = 1e-3)

||p norm where p = 1, q = 1


TypeError: __init__() missing 1 required positional argument: 'p'

## 4. Total Variation
[Back to Index](#Index)

The finite differences Operator is denoted by $\operatorname{D}$


## 4.A 1D Total Variation 
[Back to Index](#Index)

$$\|Dx\|_p$$


In [10]:
np.arange(1, 10)[4:6]


array([5, 6])

In [7]:
for p in [1, 2, 3, 5, 8, 'inf']:
    print(f'||p norm where p = {p}')
    if p == 1:
        proximal.evaluate(proximal.TV_1D_Lp_prox, proximal.TV_1D_Lp_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=1, lamb=0.5, additional_params=[p], err_thr=1e-4, plot=True)
        continue
    proximal.evaluate(proximal.TV_1D_Lp_prox, proximal.TV_1D_Lp_plus_nonneg_prox, n=10, size=(30, ), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[p], err_thr = 1e-4)

||p norm where p = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.384e-05
Average absolute error: 4.833e-07

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 6.166e-01
Average absolute error: 1.640e-01

Plotting example


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

||p norm where p = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.615e-01
Average absolute error: 2.636e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 3.373e-01
Average absolute error: 4.507e-02

||p norm where p = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.649e-01
Average absolute error: 2.005e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 2.883e-01
Average absolute error: 2.802e-02

||p norm where p = 5
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 2.432e-01
Average absolute error: 1.636e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 2.791e-01
Average absolute error: 2.081e-02

||p norm where p = 8
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 3.582e-01
Average absolute error: 1.739e-02

prox_reg(prox_nonneg(v)) IS NOT equal to 

$$\|Dx\|_p^p$$

In [8]:
for p in [1, 2, 3, 5, 8, 'inf']:
    print(f'||p^p norm where p = {p}')
    if p == 3:
        proximal.evaluate(proximal.TV_1D_Lp_prox, proximal.TV_1D_Lp_plus_nonneg_prox, n=10, size=(50, ), mean=0, sigma=1, lamb=0.5, additional_params=[p, True], err_thr=1e-4, plot=True)
        continue
    proximal.evaluate(proximal.TV_1D_Lp_prox, proximal.TV_1D_Lp_plus_nonneg_prox, n=10, size=(50, ), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[p, True], err_thr = 1e-4)

||p^p norm where p = 1
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.992e-05
Average absolute error: 5.538e-07

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 9.603e-01
Average absolute error: 1.691e-01

||p^p norm where p = 2
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 5.065e-01
Average absolute error: 5.263e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 5.573e-01
Average absolute error: 1.351e-01

||p^p norm where p = 3
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 5.897e-01
Average absolute error: 6.812e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 6.080e-01
Average absolute error: 1.286e-01

Plotting example


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

||p^p norm where p = 5
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 7.570e-01
Average absolute error: 8.439e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 5.604e-01
Average absolute error: 9.641e-02

||p^p norm where p = 8
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 6.714e-01
Average absolute error: 7.861e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 5.682e-01
Average absolute error: 8.630e-02

||p^p norm where p = inf


TypeError: '>' not supported between instances of 'str' and 'int'

### 4.B 2D Total Variation 
[Back to Index](#Index)
$$\|\operatorname{D}x\|_{p, q}$$

In [11]:
for p in [1, 2]:
    for q in [1, 2]:
        print(f'|TV*|p,q norm where p = {p} and q = {q}.')
        if p == 2 and q == 2:
            proximal.evaluate(proximal.TV_prox, proximal.TV_plus_nonneg_prox, n=10, size=(15, 15), mean=0, sigma=1, lamb=0.5, additional_params=[p, q, False], err_thr=1e-4, plot=True)
            continue
        proximal.evaluate(proximal.TV_prox, proximal.TV_plus_nonneg_prox, n=10, size=(15, 15), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[p, q, False], err_thr = 1e-3)

|TV*|p,q norm where p = 1 and q = 1.
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.719e-04
Average absolute error: 4.661e-06

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 7.084e-01
Average absolute error: 2.810e-01

|TV*|p,q norm where p = 1 and q = 2.
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.362e-01
Average absolute error: 1.373e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 2.277e-01
Average absolute error: 2.913e-02

|TV*|p,q norm where p = 2 and q = 1.
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 2.041e-01
Average absolute error: 1.292e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 6.382e-01
Average absolute error: 2.500e-01

|TV*|p,q norm where p = 2 and q = 2.
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 1.146e-01
Average abso

Button(description='Show Widgets', style=ButtonStyle())

$$\|\operatorname{D}x\|_{p, q}^{p, q}$$

In [13]:
for p in [1, 2]:
    for q in [1, 2]:
        print(f'|TV*|p,q norm where p = {p} and q = {q}.')
        if p == 2 and q == 2:
            proximal.evaluate(proximal.TV_prox, proximal.TV_plus_nonneg_prox, n=10, size=(20, 20), mean=0, sigma=1, lamb=0.5, additional_params=[p, q, True], err_thr=1e-4, plot=True)
            continue
        proximal.evaluate(proximal.TV_prox, proximal.TV_plus_nonneg_prox, n=10, size=(20, 20), mean=0, sigma=1, lamb=0.5, plot=False, additional_params=[p, q, True], err_thr = 1e-3)

|TV*|p,q norm where p = 1 and q = 1.
prox_nonneg(prox_reg(v)) SEEMS equal to prox(reg + nonneg).
Max absolute error: 1.614e-04
Average absolute error: 1.658e-06

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 6.554e-01
Average absolute error: 2.842e-01

|TV*|p,q norm where p = 1 and q = 2.
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 3.481e-01
Average absolute error: 6.100e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 8.032e-01
Average absolute error: 2.218e-01

|TV*|p,q norm where p = 2 and q = 1.
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 3.885e-01
Average absolute error: 6.586e-02

prox_reg(prox_nonneg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 5.585e-01
Average absolute error: 1.973e-01

|TV*|p,q norm where p = 2 and q = 2.
prox_nonneg(prox_reg(v)) IS NOT equal to prox(reg + nonneg)
Max absolute error: 6.485e-01
Average abso

Button(description='Show Widgets', style=ButtonStyle())

## 5. Norms on Eigendecomposition
[Back to Index](#Index)

$$\|S\|$$, 
where
$$S = \mathrm{diag}([\lambda_1, \lambda_2, ..., \lambda_i])$$

### 5.A Lp
[Back to Index](#Index)

In [3]:
for p in ['inf']:
    print(f'||p norm where p = {p}')
    proximal.evaluate(proximal.eig_prox, proximal.eig_plus_nonneg_prox, n=10, size=(20, 20), mean=0, sigma=1, lamb=0.5, additional_params=[p], err_thr = 1e-4, plot=True)

||p norm where p = inf
FATAL: Cannot solve SDPs with > 2x2 matrices without linked blas+lapack libraries
Install blas+lapack and re-compile SCS with blas+lapack library locations
ERROR: init_cone failure
Failure:could not initialize work


SolverError: Either candidate conic solvers (['ECOS']) do not support the cones output by the problem (SOC, PSD), or there are not enough constraints in the problem.

### 5.B Lp^p
[Back to Index](#Index)

## 6. Hessian Schatten Norm
[Back to Index](#Index)
$$\mathcal{R}(Lx) = \sum^N_{n = 1}\| \left[
  \begin{bmatrix}
    [D_{11}x]_n [D_{12}x]_n\\
    [D_{21}x]_n [D_{22}x]_n
  \end{bmatrix}
\right] \|_*$$

In [2]:
def Hessian_Schatten_prox(v, lamb):
    
    if len(v.shape) != 2:
        print('WARNING\n:This function is  designed for 2D operators. Terminating function.')
        return
    
    nx, ny = v.shape
    
    x = cp.Variable((nx, ny))
    
    # Defining D1 and D2x We loose one element in the *other* dimension to account for the lost element and have elements of equal size
    D1 = cp.abs((x[:, 1:] - x[:, :-1])[:-1, :])
    D2 = cp.abs((x[1:, :] - x[:-1, :])[:, :-1])
    D11 = cp.abs((D1[:, 1:] - D1[:, :-1])[:-1, :])
    D22 = cp.abs((D2[1:, :] - D2[:-1, :])[:, :-1])
    D21 = cp.abs((D2[:, 1:] - D2[:, :-1])[:-1, :])
    D12 = cp.abs((D1[1:, :] - D1[:-1, :])[:, :-1])
#     T = D11 + D22
#     D = cp.multiply(D11, D22) - cp.multiply(D12, D21)
#     lamb_1 = T/2 + cp.multiply((T)**2, cp.power(4 - D, -1))
#     lamb_2 = T/2 - cp.multiply((T)**2, cp.power(4 - D, -1))
#     y = cp.Variable((nx*ny, 2, 2))
#     y[:, 0, 0] = cp.vec(D11)
#     y[:, 0, 1] = cp.vec(D12)
#     y[:, 1, 0] = cp.vec(D21)
#     y[:, 1, 1] = cp.vec(D22)
    D11 = cp.vec(D11)
    D12 = cp.vec(D12)
    D21 = cp.vec(D21)
    D22 = cp.vec(D22)
    inner = 0
    for i in range(nx*ny):
#         y = cp.Variable((2, 2))
#         y[0, 0] = D11[i]
#         y[0, 1] = D12[i]
#         y[1, 0] = D21[i]
#         y[1, 1] = D22[i]
        a = [[D11[i], D12[i]], [D21[i], D22[i]]]
        inner = inner + cp.sum_largest(scipy.linalg.eigvals(a), 2) 
    
#     obj = cp.pnorm(lamb_1 + lamb_2, 1) + cp.sum_squares(x - v)/(2*lamb)
    obj = inner + cp.sum_squares(x - v)/(2*lamb)
    prob = cp.Problem(cp.Minimize(obj))
    prob.solve()
    
    return x.value

def Hessian_Schatten_plus_nonneg_prox(v, lamb):
    
    if len(v.shape) != 2:
        print('WARNING\n:This function is  designed for 2D operators. Terminating function.')
        return
    
    nx, ny = v.shape
    
    x = cp.Variable((nx, ny), nonneg = True)
    
    # Defining D1 and D2x We loose one element in the *other* dimension to account for the lost element and have elements of equal size
    D1 = (x[:, 1:] - x[:, :-1])[:-1, :]
    D2 = (x[1:, :] - x[:-1, :])[:, :-1]
    D11 = (D1[:, 1:] - D1[:, :-1])[:-1, :]
    D22 = (D2[1:, :] - D2[:-1, :])[:, :-1]
    D21 = (D2[:, 1:] - D2[:, :-1])[:-1, :]
    D12 = (D1[1:, :] - D1[:-1, :])[:, :-1]
#     T = D11 + D22
#     D = cp.multiply(D11, D22) - cp.multiply(D12, D21)
#     lamb_1 = T/2 + cp.multiply((T)**2, cp.power(4 - D, -1))
#     lamb_2 = T/2 - cp.multiply((T)**2, cp.power(4 - D, -1))
#     y = cp.Variable((nx*ny, 2, 2))
#     y[:, 0, 0] = cp.vec(D11)
#     y[:, 0, 1] = cp.vec(D12)
#     y[:, 1, 0] = cp.vec(D21)
#     y[:, 1, 1] = cp.vec(D22)
    D11 = cp.vec(D11)
    D12 = cp.vec(D12)
    D21 = cp.vec(D21)
    D22 = cp.vec(D22)
    inner = 0
    for i in range(nx*ny):
#         y = cp.Variable((2, 2))
#         y[0, 0] = D11[i]
#         y[0, 1] = D12[i]
#         y[1, 0] = D21[i]
#         y[1, 1] = D22[i]
        a = [[D11[i], D12[i]], [D21[i], D22[i]]]
        eig = scipy.linalg.eigvals(a)
        inner = inner + eig[0] + eig[1]# cp.sum_largest(scipy.linalg.eigvals(a), 2) 
    
#     obj = cp.pnorm(lamb_1 + lamb_2, 1) + cp.sum_squares(x - v)/(2*lamb)
    obj = inner + cp.sum_squares(x - v)/(2*lamb)

    
    prob = cp.Problem(cp.Minimize(obj))
    prob.solve()
    
    return x.value

# <a name = 'Eval'></a> Evaluation

The following function `evaluate` is designed to evaluate a number of random vectors created from normal distributions on the cases presented in [section 1](#Intro):

$$\mathrm{prox}_{L^2 + \delta_{\rm I\!R_+^N }} $$
$$ \mathrm{prox}_{\mathrm{L}^2}(\mathrm{prox}_{\delta_{\rm I\!R_+^N}})$$ 
$$\mathrm{prox}_{\delta_{\rm I\!R_+^N}}(\mathrm{prox}_{\mathcal{R}})$$

And searches for equality. It takes as input parameters:
* `reg` (function): Closed-form solution of the proximal operator of the regularizer, **Considerar como input problema CVXPY**
* `reg_nonneg` (function): Closed-form solution of the proximal operator of the regularizer + $\delta_{\rm I\!R_+^N }$,
* `n` (int): Number of vectors
* `shape` (tuple): Shape of the vectors
* `mean` (scalar): Mean of the vectors
* `sigma` (scalar): $\sigma$ of the normal distribution used to sample the vectors
* `lamb` (scalar): Parameter $\lambda$ to be used in the pointwise operations.

It returns:
* `reg_nonneg` (boolean): The findings on whether $\mathrm{prox}_{L^2 + \delta_{\rm I\!R_+^N }} == \mathrm{prox}_{\mathrm{L}^2}(\mathrm{prox}_{\delta_{\rm I\!R_+^N}})$
* `nonneg_reg` (boolean): The findings on whether
$\mathrm{prox}_{L^2 + \delta_{\rm I\!R_+^N }} == \mathrm{prox}_{\delta_{\rm I\!R_+^N}}(\mathrm{prox}_{\Re})$

Now we will run the function for the L1 and L2 norm

In [ ]:
# print('L2 Norm:')
# proximal.evaluate(l2_prox, l2_plus_nonneg_prox, n=100, size=(100,), mean=0, sigma=1, lamb=3, plot=True,)
# print('\nL1 Norm:')
# proximal.evaluate(l1_prox, l1_plus_nonneg_prox, n=100, size=(100,), mean=0, sigma=1, lamb=1, plot=True )
# print('\n||Qx||1 Norm, where Q is the DCT:')
# proximal.evaluate(DCT_l1_prox, DCT_l1_plus_nonneg, n=100, size=(100,), mean=0, sigma=1, lamb=0.1)
# print('\n||Qx||1 Norm, where Q is the finite differences:')
# proximal.evaluate(Q_L1_prox_1D, Q_L1_plus_nonneg_prox_1D, n=100, size=(50,), mean=0, sigma=1, lamb=0.5, plot = True,
#                   rtol=1e-2, atol=1e-3)
# print('\n$||x||_0$ Norm')
# proximal.evaluate(l0_prox, l0_plus_nonneg_prox, n=10, size=(50,), mean=0, sigma=1, lamb=1, plot = True,
#                   rtol=1e-2, atol=1e-3)
print('\nTV 2,1 Regularizer:')
proximal.evaluate(TV_prox, TV_plus_nonneg_prox, n=2, size=(20, 20), mean=0, sigma=1, lamb=0.2, plot = True, rtol=1e-3, atol=1e-3)
# print('\nTV(x) Regularizer:')
# proximal.evaluate(Hessian_Schatten_prox, Hessian_Schatten_plus_nonneg_prox, n=10, size=(20, 20), mean=0, sigma=1, lamb=0.2, plot = True, rtol=1e-3, atol=1e-3)


## References

[1.](https://arc.aiaa.org/doi/pdf/10.2514/1.26320?casa_token=EpyiTfodqo4AAAAA%3AGmnFHyrImbNxMMk2ONs1c9wpN6bLTip8_a7irQweswoms0vMBtL1kGu6h8v6IK76_zhhhtk0KPA&) Pol del Aguila Pla and Joakim Jaldén, Cell detection by functional inverse diffusion and non-negative group sparsity—Part II: Proximal optimization and Performance evaluation, IEEE Transactions on Signal Processing, vol. 66, no. 20, pp. 5422–5437, 2018

[2.](http://bigwww.epfl.ch/publications/soubies1904.pdf) E. Soubies, F. Soulez, M.T. McCann, T.-a. Pham, L. Donati, T. Debarre, D. Sage, M. Unser, "Pocket Guide to Solve Inverse Problems with GlobalBioIm," Inverse Problems, vol. 35, no. 10, paper no. 104006, pp. 1-20, October 2019